In [2]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [3]:
standings = pd.read_csv('standings.csv')
standings

,standing_id,season_id,league_id,position,team_id,played_games,won,draw,lost,points,goals_for,goals_against,goal_difference,form
0,21,4,4,1,3,34,28,6,0,90,89,24,65,"['D', 'D', 'W', 'W', 'W']"
1,22,4,4,2,10,34,23,4,7,73,78,39,39,"['L', 'D', 'W', 'W', 'W']"
2,23,4,4,3,5,34,23,3,8,72,94,45,49,"['D', 'L', 'L', 'W', 'L']"
3,24,4,4,4,721,34,19,8,7,65,77,39,38,"['W', 'W', 'D', 'D', 'D']"
4,25,4,4,5,4,34,18,9,7,63,68,43,25,"['W', 'W', 'W', 'L', 'W']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,94,5,5,14,543,34,9,6,19,33,30,55,-25,"['L', 'D', 'D', 'L', 'L']"
92,95,5,5,15,533,34,7,11,16,32,34,45,-11,"['L', 'D', 'W', 'L', 'L']"
93,96,5,5,16,545,34,8,5,21,29,35,58,-23,"['W', 'L', 'L', 'L', 'L']"
94,97,5,5,17,525,34,7,8,19,29,43,66,-23,"['L', 'L', 'L', 'L', 'W']"


In [4]:
players = pd.read_csv('players.csv')
merged_data = standings.merge(players, on='team_id')
merged_data


,standing_id,season_id,league_id,position_x,team_id,played_games,won,draw,lost,points,goals_for,goals_against,goal_difference,form,player_id,name,position_y,date_of_birth,nationality
0,21,4,4,1,3,34,28,6,0,90,89,24,65,"['D', 'D', 'W', 'W', 'W']",165,Niklas Lomb,Goalkeeper,1993-07-28,Germany
1,21,4,4,1,3,34,28,6,0,90,89,24,65,"['D', 'D', 'W', 'W', 'W']",6696,Lukáš Hrádecký,Goalkeeper,1989-11-24,Finland
2,21,4,4,1,3,34,28,6,0,90,89,24,65,"['D', 'D', 'W', 'W', 'W']",136350,Matej Kovar,Goalkeeper,2000-05-17,Czech Republic
3,21,4,4,1,3,34,28,6,0,90,89,24,65,"['D', 'D', 'W', 'W', 'W']",149,Jonathan Tah,Defence,1996-02-11,Germany
4,21,4,4,1,3,34,28,6,0,90,89,24,65,"['D', 'D', 'W', 'W', 'W']",8148,Timothy Fosu-Mensah,Defence,1998-01-02,Netherlands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,98,5,5,18,541,34,5,10,19,25,26,60,-34,"['L', 'W', 'L', 'L', 'L']",32315,Jérémie Bela,Offence,1993-04-08,Angola
3146,98,5,5,18,541,34,5,10,19,25,26,60,-34,"['L', 'W', 'L', 'L', 'L']",84562,Shamar Nicholson,Offence,1997-03-16,Jamaica
3147,98,5,5,18,541,34,5,10,19,25,26,60,-34,"['L', 'W', 'L', 'L', 'L']",101297,Jim Allevinah,Offence,1995-02-27,Gabon
3148,98,5,5,18,541,34,5,10,19,25,26,60,-34,"['L', 'W', 'L', 'L', 'L']",150934,Alan Virginius,Offence,2003-01-03,France


In [5]:
filtered_data = merged_data[['won','position_y', 'team_id', 'nationality']]
filtered_data


,won,position_y,team_id,nationality
0,28,Goalkeeper,3,Germany
1,28,Goalkeeper,3,Finland
2,28,Goalkeeper,3,Czech Republic
3,28,Defence,3,Germany
4,28,Defence,3,Netherlands
...,...,...,...,...
3145,5,Offence,541,Angola
3146,5,Offence,541,Jamaica
3147,5,Offence,541,Gabon
3148,5,Offence,541,France


In [6]:
pivot_nat = filtered_data.pivot_table(index='won', columns='nationality', aggfunc='size', fill_value=0)
df=pivot_nat


In [15]:
df = df.reset_index()  # Reset the index to make 'won' a column
y = df['won'].values.reshape(-1, 1)
X = df.drop(columns=['won']).values
df

nationality,index,won,Albania,Algeria,Andorra,Angola,Argentina,Armenia,Australia,Austria,...,Tunisia,Turkey,USA,Ukraine,Uruguay,Uzbekistan,Venezuela,Wales,Zambia,Zimbabwe
0,0,2,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,0,0,0,0,2,0,0,3,...,1,0,1,0,0,0,0,2,0,0
2,2,4,1,0,0,0,2,0,0,0,...,0,0,0,0,3,0,0,0,0,0
3,3,5,0,1,0,1,0,1,0,3,...,0,0,1,0,0,0,0,0,0,0
4,4,6,0,0,0,0,6,0,0,0,...,0,0,0,0,2,0,1,2,0,2
5,5,7,2,1,0,0,0,1,1,5,...,1,0,3,1,0,0,0,0,0,0
6,6,8,6,3,0,1,4,0,1,0,...,1,1,0,0,4,1,0,0,2,0
7,7,9,3,0,0,0,3,0,0,1,...,1,0,3,1,0,0,0,2,0,1
8,8,10,1,0,0,0,9,0,0,4,...,2,1,3,1,2,0,0,0,0,0
9,9,11,1,3,0,0,2,0,1,6,...,1,0,2,0,0,0,1,0,0,0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 219.11944371712377
R-squared: -4.586614712334599


In [13]:
# Input for a new row
new_data = {}
for nationality in df.columns[1:]:  # Skip the 'won' column
    while True:
        try:
            value = input(f"Enter the number of players from {nationality}: ")
            new_data[nationality] = int(value)
            break
        except ValueError:
            print(f"Invalid input '{value}'. Please enter a valid integer.")

# Convert the input into a numpy array
new_row = np.array([list(new_data.values())])

# Predict the number of goals
predicted_goals = model.predict(new_row)
print(f"Predicted number of goals: {predicted_goals[0][0]}")

Invalid input ''. Please enter a valid integer.
Invalid input ''. Please enter a valid integer.
Invalid input ''. Please enter a valid integer.
Invalid input ''. Please enter a valid integer.
Invalid input '-'. Please enter a valid integer.
Invalid input ''. Please enter a valid integer.
Invalid input ''. Please enter a valid integer.
Predicted number of goals: 31.854373101364978
